In [ ]:

# import warnings
# warnings.filterwarnings('ignore')

# import pandas as pd
# import numpy as np
# import pickle
# import json
# from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
# from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, LabelEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Optional imports
# try:
#     from imblearn.over_sampling import SMOTE
#     from imblearn.pipeline import Pipeline as ImbPipeline
#     IMBLEARN_AVAILABLE = True
# except Exception:
#     IMBLEARN_AVAILABLE = False

# try:
#     from xgboost import XGBClassifier
#     XGBOOST_AVAILABLE = True
# except Exception:
#     XGBOOST_AVAILABLE = False

# def load_and_enhance_data(file_path):
#     """Load data and create enhanced features"""
#     df = pd.read_csv('accident-project.csv')
#     df.columns = df.columns.str.strip()
    
#     # Clean data
#     df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    
#     # Enhanced time features
#     if 'Time' in df.columns:
#         df['Time_parsed'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
#         df.loc[df['Time_parsed'].isna(), 'Time_parsed'] = pd.to_datetime(
#             df.loc[df['Time_parsed'].isna(), 'Time'], format='%H:%M', errors='coerce'
#         )
#         df['Hour_of_Day'] = df['Time_parsed'].dt.hour
#         df['Time_of_Day'] = pd.cut(df['Hour_of_Day'], 
#                                  bins=[0, 6, 12, 18, 24], 
#                                  labels=['Night', 'Morning', 'Afternoon', 'Evening'],
#                                  include_lowest=True)
#         df = df.drop(columns=['Time', 'Time_parsed'], errors='ignore')
    
#     # Create accident severity risk score
#     risk_factors = []
#     if 'Number_of_vehicles_involved' in df.columns:
#         risk_factors.append(df['Number_of_vehicles_involved'])
#     if 'Number_of_casualties' in df.columns:
#         risk_factors.append(df['Number_of_casualties'])
    
#     if risk_factors:
#         df['Risk_Score'] = sum(risk_factors) / len(risk_factors)
    
#     # Enhanced driver experience categorization
#     if 'Driving_experience' in df.columns:
#         df['Experience_Level'] = df['Driving_experience'].map({
#             'No Licence': 'Novice',
#             'Below 1yr': 'Novice', 
#             '1-2yr': 'Beginner',
#             '2-5yr': 'Intermediate',
#             '5-10yr': 'Experienced',
#             'Above 10yr': 'Expert',
#             'Unknown': 'Unknown'
#         })
    
#     return df

# def create_preprocessor(numeric_features, ordinal_cols, nominal_cols, ordinal_categories):
#     """Create preprocessing pipeline"""
#     numeric_transformer = Pipeline([
#         ('imputer', SimpleImputer(strategy='median')),
#         ('scaler', StandardScaler())
#     ])

#     ordinal_transformer = Pipeline([
#         ('imputer', SimpleImputer(strategy='most_frequent')),
#         ('encoder', OrdinalEncoder(categories=ordinal_categories, 
#                                   handle_unknown='use_encoded_value', unknown_value=-1))
#     ])

#     nominal_transformer = Pipeline([
#         ('imputer', SimpleImputer(strategy='most_frequent')),
#         ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
#     ])

#     preprocessor = ColumnTransformer(
#         transformers=[
#             ('num', numeric_transformer, numeric_features),
#             ('ord', ordinal_transformer, ordinal_cols),
#             ('cat', nominal_transformer, nominal_cols)
#         ],
#         remainder='drop'
#     )
    
#     return preprocessor

# def train_models():
#     """Main training function"""
#     print("🚀 Starting Model Training...")
    
#     # Load and prepare data
#     file_path = "accident-project.csv"
#     df = load_and_enhance_data(file_path)
    
#     # Drop duplicates and missing target
#     df = df.drop_duplicates()
#     target_col = "Accident_severity"
#     df = df[~df[target_col].isna()].copy()
    
#     print(f"📊 Dataset shape: {df.shape}")
#     print(f"🎯 Target distribution:\n{df[target_col].value_counts()}")
    
#     # Feature groups
#     numeric_features = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
#     categorical_features = df.select_dtypes(include=['object']).columns.tolist()
    
#     # Remove target
#     numeric_features = [c for c in numeric_features if c != target_col]
#     categorical_features = [c for c in categorical_features if c != target_col]
    
#     # Add engineered features
#     if 'Hour_of_Day' in df.columns and 'Hour_of_Day' not in numeric_features:
#         numeric_features.append('Hour_of_Day')
#     if 'Risk_Score' in df.columns and 'Risk_Score' not in numeric_features:
#         numeric_features.append('Risk_Score')
    
#     # Ordinal features with proper ordering
#     ordinal_mappings = {
#         'Day_of_week': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
#         'Age_band_of_driver': ['Under 18', '18-30', '31-50', 'Over 51', 'Unknown'],
#         'Educational_level': ['Illiterate', 'Writing & reading', 'Elementary school', 
#                              'Junior high school', 'High school', 'Above high school', 'Unknown'],
#         'Driving_experience': ['No Licence', 'Below 1yr', '1-2yr', '2-5yr', '5-10yr', 'Above 10yr', 'Unknown'],
#         'Service_year_of_vehicle': ['Below 1yr', '1-2yr', '2-5yr', '5-10yr', 'Above 10yr', 'Unknown'],
#         'Experience_Level': ['Novice', 'Beginner', 'Intermediate', 'Experienced', 'Expert', 'Unknown']
#     }
    
#     ordinal_cols = [col for col in ordinal_mappings.keys() if col in df.columns]
#     ordinal_categories = [ordinal_mappings[col] for col in ordinal_cols]
    
#     # Nominal features
#     nominal_cols = [c for c in categorical_features if c not in ordinal_cols]
    
#     print(f"🔧 Processing {len(numeric_features)} numeric, {len(ordinal_cols)} ordinal, {len(nominal_cols)} nominal features")
    
#     # Create preprocessor
#     preprocessor = create_preprocessor(numeric_features, ordinal_cols, nominal_cols, ordinal_categories)
    
#     # Prepare target
#     le_target = LabelEncoder()
#     y = le_target.fit_transform(df[target_col])
#     X = df.drop(columns=[target_col])
    
#     target_mapping = dict(zip(le_target.classes_, le_target.transform(le_target.classes_)))
#     print("🎯 Target mapping:", target_mapping)
    
#     # Train-test split
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.2, stratify=y, random_state=42
#     )
    
#     # Define models
#     models = {
#         'Logistic Regression': LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000),
#         'Decision Tree': DecisionTreeClassifier(random_state=42, class_weight='balanced'),
#         'Random Forest': RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=100),
#         'Gradient Boosting': GradientBoostingClassifier(random_state=42),
#         'SVM': SVC(random_state=42, class_weight='balanced', probability=True)
#     }
    
#     if XGBOOST_AVAILABLE:
#         models['XGBoost'] = XGBClassifier(random_state=42, eval_metric='mlogloss')
    
#     # Train models
#     best_models = {}
#     results = {}
    
#     print("\n🔮 Training Models...")
#     for name, model in models.items():
#         print(f"Training {name}...")
        
#         # Create pipeline
#         if IMBLEARN_AVAILABLE:
#             pipeline = ImbPipeline([
#                 ('preprocessor', preprocessor),
#                 ('smote', SMOTE(random_state=42)),
#                 ('clf', model)
#             ])
#         else:
#             pipeline = Pipeline([
#                 ('preprocessor', preprocessor),
#                 ('clf', model)
#             ])
        
#         # Train model
#         pipeline.fit(X_train, y_train)
        
#         # Evaluate
#         y_pred = pipeline.predict(X_test)
#         metrics = {
#             'Accuracy': accuracy_score(y_test, y_pred),
#             'Precision': precision_score(y_test, y_pred, average='weighted', zero_division=0),
#             'Recall': recall_score(y_test, y_pred, average='weighted', zero_division=0),
#             'F1-Score': f1_score(y_test, y_pred, average='weighted', zero_division=0)
#         }
        
#         best_models[name] = pipeline
#         results[name] = metrics
        
#         print(f"  ✅ {name}: F1 = {metrics['F1-Score']:.4f}")
    
#     # Select best model
#     best_model_name = max(results, key=lambda x: results[x]['F1-Score'])
#     best_model = best_models[best_model_name]
    
#     print(f"\n🏆 BEST MODEL: {best_model_name}")
#     print(f"📊 Performance: {results[best_model_name]}")
    
#     # Save model artifacts
#     model_artifacts = {
#         'model': best_model,
#         'preprocessor': preprocessor,
#         'label_encoder': le_target,
#         'target_mapping': target_mapping,
#         'feature_names': {
#             'numeric': numeric_features,
#             'ordinal': ordinal_cols,
#             'nominal': nominal_cols
#         },
#         'model_performance': results,
#         'best_model_name': best_model_name,
#         'all_models': best_models
#     }
    
#     # Save with pickle
#     with open('accident_severity_model.pkl', 'wb') as f:
#         pickle.dump(model_artifacts, f)
    
#     # Save metadata
#     metadata = {
#         'model_info': {
#             'best_model': best_model_name,
#             'performance': results[best_model_name],
#             'target_classes': le_target.classes_.tolist(),
#             'feature_count': len(numeric_features) + len(ordinal_cols) + len(nominal_cols)
#         },
#         'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
#         'dataset_info': {
#             'total_samples': len(df),
#             'training_samples': len(X_train),
#             'test_samples': len(X_test)
#         }
#     }
    
#     with open('model_metadata.json', 'w') as f:
#         json.dump(metadata, f, indent=2)
    
#     print("\n✅ Model training completed successfully!")
#     print("💾 Model saved as 'accident_severity_model.pkl'")
#     print("📋 Metadata saved as 'model_metadata.json'")
    
#     # Create sample visualization
#     plt.figure(figsize=(10, 6))
#     models_list = list(results.keys())
#     f1_scores = [results[model]['F1-Score'] for model in models_list]
    
#     plt.barh(models_list, f1_scores, color='skyblue')
#     plt.xlabel('F1-Score')
#     plt.title('Model Performance Comparison')
#     plt.tight_layout()
#     plt.savefig('model_comparison.png', dpi=300, bbox_inches='tight')
#     print("📈 Visualization saved as 'model_comparison.png'")
    
#     return model_artifacts, metadata

# if __name__ == "__main__":
#     train_models()

In [ ]:
# import warnings
# warnings.filterwarnings('ignore')

# import pandas as pd
# import numpy as np
# import pickle
# import json
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
# from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, LabelEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
# import plotly.express as px
# import plotly.graph_objects as go

In [ ]:


# # Optional imports
# try:
#     from imblearn.over_sampling import SMOTE
#     from imblearn.pipeline import Pipeline as ImbPipeline
#     IMBLEARN_AVAILABLE = True
# except Exception:
#     IMBLEARN_AVAILABLE = False

# try:
#     from xgboost import XGBClassifier
#     XGBOOST_AVAILABLE = True
# except Exception:
#     XGBOOST_AVAILABLE = False

# class AccidentSeverityPredictor:
#     def __init__(self):
#         self.artifacts = {}
#         self.metadata = {}
        
#     def load_and_enhance_data(self, file_path):
#         """Load data and create enhanced features"""
#         print("📂 Loading dataset...")
#         df = pd.read_csv(file_path)
#         df.columns = df.columns.str.strip()
        
#         # Clean data - handle 'na' values and whitespace
#         df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
#         df = df.replace('na', np.nan)
#         df = df.replace('unknown', np.nan)
#         df = df.replace('Unknown', np.nan)
        
#         # Enhanced time features
#         if 'Time' in df.columns:
#             print("⏰ Processing time features...")
#             # Handle different time formats
#             df['Time_parsed'] = pd.to_datetime(df['Time'], format='%H:%M:%S', errors='coerce')
#             mask = df['Time_parsed'].isna()
#             df.loc[mask, 'Time_parsed'] = pd.to_datetime(df.loc[mask, 'Time'], format='%H:%M', errors='coerce')
            
#             df['Hour_of_Day'] = df['Time_parsed'].dt.hour.fillna(12)  # Default to noon if missing
#             df['Time_of_Day'] = pd.cut(df['Hour_of_Day'], 
#                                      bins=[0, 6, 12, 18, 24], 
#                                      labels=['Night', 'Morning', 'Afternoon', 'Evening'],
#                                      include_lowest=True)
#             df = df.drop(columns=['Time', 'Time_parsed'], errors='ignore')
        
#         # Create accident severity risk score
#         risk_factors = []
#         if 'Number_of_vehicles_involved' in df.columns:
#             risk_factors.append(df['Number_of_vehicles_involved'])
#         if 'Number_of_casualties' in df.columns:
#             risk_factors.append(df['Number_of_casualties'])
        
#         if risk_factors:
#             df['Risk_Score'] = sum(risk_factors) / len(risk_factors)
#         else:
#             df['Risk_Score'] = 1.0  # Default risk score
        
#         # Enhanced driver experience categorization
#         if 'Driving_experience' in df.columns:
#             df['Experience_Level'] = df['Driving_experience'].map({
#                 'No Licence': 'Novice',
#                 'Below 1yr': 'Novice', 
#                 '1-2yr': 'Beginner',
#                 '2-5yr': 'Intermediate',
#                 '5-10yr': 'Experienced',
#                 'Above 10yr': 'Expert',
#                 'Unknown': 'Unknown',
#                 'nan': 'Unknown'
#             }).fillna('Unknown')
        
#         print(f"✅ Enhanced dataset with new features. Shape: {df.shape}")
#         return df

#     def analyze_dataset(self, df, target_col):
#         """Comprehensive dataset analysis"""
#         print("\n🔍 Dataset Analysis:")
#         print(f"   Total samples: {len(df)}")
#         print(f"   Total features: {len(df.columns)}")
#         print(f"   Target distribution:\n{df[target_col].value_counts()}")
        
#         # Missing values analysis
#         missing = df.isnull().sum()
#         if missing.sum() > 0:
#             print(f"   Missing values: {missing[missing > 0].to_dict()}")
        
#         return df

#     def create_preprocessor(self, numeric_features, ordinal_cols, nominal_cols, ordinal_categories):
#         """Create advanced preprocessing pipeline"""
#         print("🔧 Creating preprocessing pipeline...")
        
#         numeric_transformer = Pipeline([
#             ('imputer', SimpleImputer(strategy='median')),
#             ('scaler', StandardScaler())
#         ])

#         ordinal_transformer = Pipeline([
#             ('imputer', SimpleImputer(strategy='most_frequent')),
#             ('encoder', OrdinalEncoder(categories=ordinal_categories, 
#                                       handle_unknown='use_encoded_value', unknown_value=-1))
#         ])

#         nominal_transformer = Pipeline([
#             ('imputer', SimpleImputer(strategy='most_frequent')),
#             ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
#         ])

#         preprocessor = ColumnTransformer(
#             transformers=[
#                 ('num', numeric_transformer, numeric_features),
#                 ('ord', ordinal_transformer, ordinal_cols),
#                 ('cat', nominal_transformer, nominal_cols)
#             ],
#             remainder='drop'
#         )
        
#         return preprocessor

#     def train_models(self, file_path="accident-project.csv"):
#         """Main training function"""
#         print("🚀 Starting Advanced Model Training...")
        
#         # Load and prepare data
#         target_col = "Accident_severity"
#         df = self.load_and_enhance_data(file_path)
        
#         # Basic cleaning
#         df = df.drop_duplicates()
#         df = df[~df[target_col].isna()].copy()
        
#         # Analyze dataset
#         df = self.analyze_dataset(df, target_col)
        
#         # Feature groups - using actual columns from your dataset
#         numeric_features = ['Number_of_vehicles_involved', 'Number_of_casualties', 'Hour_of_Day', 'Risk_Score']
#         numeric_features = [col for col in numeric_features if col in df.columns]
        
#         # Define categorical features based on common accident dataset columns
#         categorical_features = ['Day_of_week', 'Age_band_of_driver', 'Sex_of_driver', 
#                                'Educational_level', 'Driving_experience', 'Type_of_vehicle',
#                                'Area_accident_occured', 'Road_surface_conditions', 
#                                'Light_conditions', 'Weather_conditions', 'Time_of_Day',
#                                'Experience_Level']
#         categorical_features = [col for col in categorical_features if col in df.columns]
        
#         # Remove target if it sneaks in
#         numeric_features = [c for c in numeric_features if c != target_col]
#         categorical_features = [c for c in categorical_features if c != target_col]
        
#         print(f"🔧 Processing {len(numeric_features)} numeric, {len(categorical_features)} categorical features")
        
#         # Ordinal features with proper ordering
#         ordinal_mappings = {
#             'Day_of_week': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
#             'Age_band_of_driver': ['Under 18', '18-30', '31-50', 'Over 51'],
#             'Educational_level': ['Illiterate', 'Elementary school', 'Junior high school', 
#                                  'High school', 'Above high school'],
#             'Driving_experience': ['No Licence', 'Below 1yr', '1-2yr', '2-5yr', '5-10yr', 'Above 10yr'],
#             'Experience_Level': ['Novice', 'Beginner', 'Intermediate', 'Experienced', 'Expert']
#         }
        
#         # Only use ordinal mappings for columns that exist in dataset
#         ordinal_cols = [col for col in ordinal_mappings.keys() if col in df.columns]
#         ordinal_categories = [ordinal_mappings[col] for col in ordinal_cols]
        
#         # Nominal features are the rest
#         nominal_cols = [c for c in categorical_features if c not in ordinal_cols]
        
#         print(f"   - Ordinal features: {ordinal_cols}")
#         print(f"   - Nominal features: {nominal_cols}")
        
#         # Create preprocessor
#         preprocessor = self.create_preprocessor(numeric_features, ordinal_cols, nominal_cols, ordinal_categories)
        
#         # Prepare target
#         le_target = LabelEncoder()
#         y = le_target.fit_transform(df[target_col])
#         X = df.drop(columns=[target_col])
        
#         target_mapping = dict(zip(le_target.classes_, le_target.transform(le_target.classes_)))
#         print(f"🎯 Target classes: {target_mapping}")
        
#         # Train-test split
#         X_train, X_test, y_train, y_test = train_test_split(
#             X, y, test_size=0.2, stratify=y, random_state=42
#         )
        
#         print(f"📊 Train set: {X_train.shape}, Test set: {X_test.shape}")
        
#         # Define models with optimized parameters
#         models = {
#             'Logistic Regression': LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000),
#             'Decision Tree': DecisionTreeClassifier(random_state=42, class_weight='balanced', max_depth=10),
#             'Random Forest': RandomForestClassifier(random_state=42, class_weight='balanced', 
#                                                   n_estimators=200, max_depth=15),
#             'Gradient Boosting': GradientBoostingClassifier(random_state=42, n_estimators=100, max_depth=6),
#             'SVM': SVC(random_state=42, class_weight='balanced', probability=True, kernel='rbf')
#         }
        
#         if XGBOOST_AVAILABLE:
#             models['XGBoost'] = XGBClassifier(random_state=42, eval_metric='mlogloss', 
#                                             n_estimators=200, max_depth=6)
        
#         # Train and evaluate models
#         best_models = {}
#         results = {}
#         feature_importances = {}
        
#         print("\n🔮 Training and Evaluating Models...")
#         for name, model in models.items():
#             print(f"   Training {name}...")
            
#             # Create pipeline
#             if IMBLEARN_AVAILABLE:
#                 pipeline = ImbPipeline([
#                     ('preprocessor', preprocessor),
#                     ('smote', SMOTE(random_state=42)),
#                     ('clf', model)
#                 ])
#             else:
#                 pipeline = Pipeline([
#                     ('preprocessor', preprocessor),
#                     ('clf', model)
#                 ])
            
#             # Train model
#             pipeline.fit(X_train, y_train)
            
#             # Evaluate
#             y_pred = pipeline.predict(X_test)
#             y_proba = pipeline.predict_proba(X_test)
            
#             metrics = {
#                 'Accuracy': accuracy_score(y_test, y_pred),
#                 'Precision': precision_score(y_test, y_pred, average='weighted', zero_division=0),
#                 'Recall': recall_score(y_test, y_pred, average='weighted', zero_division=0),
#                 'F1-Score': f1_score(y_test, y_pred, average='weighted', zero_division=0)
#             }
            
#             best_models[name] = pipeline
#             results[name] = metrics
            
#             # Store feature importance if available
#             if hasattr(model, 'feature_importances_'):
#                 try:
#                     # Get feature names after preprocessing
#                     feature_names = self.get_feature_names(pipeline.named_steps['preprocessor'])
#                     importances = model.feature_importances_
#                     feature_importances[name] = {
#                         'features': feature_names,
#                         'importances': importances
#                     }
#                 except Exception as e:
#                     print(f"      Could not extract feature importance for {name}: {e}")
            
#             print(f"      ✅ {name}: F1 = {metrics['F1-Score']:.4f}, Accuracy = {metrics['Accuracy']:.4f}")
        
#         # Select best model
#         best_model_name = max(results, key=lambda x: results[x]['F1-Score'])
#         best_model = best_models[best_model_name]
        
#         print(f"\n🏆 BEST MODEL: {best_model_name}")
#         print(f"📊 Performance: {results[best_model_name]}")
        
#         # Generate comprehensive report
#         self.generate_model_report(best_model, X_test, y_test, le_target, best_model_name)
        
#         # Save model artifacts
#         self.save_model_artifacts(best_models, results, preprocessor, le_target, target_mapping,
#                                  numeric_features, ordinal_cols, nominal_cols, best_model_name,
#                                  feature_importances, df, X_train, X_test, y_train, y_test)
        
#         return best_models[best_model_name]

#     def get_feature_names(self, column_transformer):
#         """Get feature names after preprocessing"""
#         feature_names = []
        
#         # Numeric features
#         for name, transformer, features in column_transformer.transformers_:
#             if name == 'num':
#                 feature_names.extend(features)
#             elif name == 'ord':
#                 feature_names.extend(features)
#             elif name == 'cat':
#                 if hasattr(transformer.named_steps['encoder'], 'get_feature_names_out'):
#                     cat_features = transformer.named_steps['encoder'].get_feature_names_out(features)
#                     feature_names.extend(cat_features)
#                 else:
#                     feature_names.extend(features)
        
#         return feature_names

#     def generate_model_report(self, model, X_test, y_test, le_target, model_name):
#         """Generate comprehensive model performance report"""
#         print("\n📋 Generating Model Performance Report...")
        
#         y_pred = model.predict(X_test)
#         y_proba = model.predict_proba(X_test)
        
#         # Classification report
#         print("\n📊 Classification Report:")
#         print(classification_report(y_test, y_pred, target_names=le_target.classes_))
        
#         # Confusion matrix
#         cm = confusion_matrix(y_test, y_pred)
#         print(f"\n🎯 Confusion Matrix:\n{cm}")
        
#         # Create visualization
#         self.create_performance_visualizations(cm, le_target.classes_, model_name)

#     def create_performance_visualizations(self, cm, class_names, model_name):
#         """Create performance visualizations"""
#         try:
#             # Confusion matrix heatmap
#             plt.figure(figsize=(8, 6))
#             sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
#                        xticklabels=class_names, yticklabels=class_names)
#             plt.title(f'Confusion Matrix - {model_name}')
#             plt.xlabel('Predicted')
#             plt.ylabel('Actual')
#             plt.tight_layout()
#             plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
#             plt.close()
            
#             print("✅ Performance visualizations saved")
#         except Exception as e:
#             print(f"⚠️  Could not create visualizations: {e}")

#     def save_model_artifacts(self, best_models, results, preprocessor, le_target, target_mapping,
#                            numeric_features, ordinal_cols, nominal_cols, best_model_name,
#                            feature_importances, df, X_train, X_test, y_train, y_test):
#         """Save all model artifacts"""
#         print("\n💾 Saving model artifacts...")
        
#         best_model = best_models[best_model_name]
        
#         model_artifacts = {
#             'model': best_model,
#             'all_models': best_models,
#             'preprocessor': preprocessor,
#             'label_encoder': le_target,
#             'target_mapping': target_mapping,
#             'feature_names': {
#                 'numeric': numeric_features,
#                 'ordinal': ordinal_cols,
#                 'nominal': nominal_cols
#             },
#             'model_performance': results,
#             'best_model_name': best_model_name,
#             'feature_importances': feature_importances,
#             'dataset_info': {
#                 'train_shape': X_train.shape,
#                 'test_shape': X_test.shape,
#                 'target_distribution': dict(df['Accident_severity'].value_counts())
#             }
#         }
        
#         # Save with pickle
#         with open('accident_severity_model.pkl', 'wb') as f:
#             pickle.dump(model_artifacts, f)
        
#         # Save metadata
#         metadata = {
#             'model_info': {
#                 'best_model': best_model_name,
#                 'performance': results[best_model_name],
#                 'target_classes': le_target.classes_.tolist(),
#                 'feature_count': len(numeric_features) + len(ordinal_cols) + len(nominal_cols),
#                 'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
#             },
#             'dataset_info': {
#                 'total_samples': len(df),
#                 'training_samples': len(X_train),
#                 'test_samples': len(X_test),
#                 'feature_breakdown': {
#                     'numeric': len(numeric_features),
#                     'ordinal': len(ordinal_cols),
#                     'nominal': len(nominal_cols)
#                 }
#             },
#             'model_comparison': results
#         }
        
#         with open('model_metadata.json', 'w') as f:
#             json.dump(metadata, f, indent=2)
        
#         print("✅ Model artifacts saved successfully!")
#         print(f"📁 Files created:")
#         print(f"   - accident_severity_model.pkl")
#         print(f"   - model_metadata.json") 
#         print(f"   - confusion_matrix.png")
        
#         # Print final summary
#         print(f"\n🎉 TRAINING COMPLETED SUCCESSFULLY!")
#         print(f"🏆 Best Model: {best_model_name}")
#         print(f"📈 Best F1-Score: {results[best_model_name]['F1-Score']:.4f}")
#         print(f"🎯 Best Accuracy: {results[best_model_name]['Accuracy']:.4f}")

# def main():
#     """Main execution function"""
#     try:
#         predictor = AccidentSeverityPredictor()
#         model = predictor.train_models("accident-project.csv")
        
#         print("\n🚀 Next steps:")
#         print("   1. Run: streamlit run enhanced_streamlit_app.py")
#         print("   2. Open http://localhost:8501 in your browser")
#         print("   3. Start making predictions!")
        
#     except Exception as e:
#         print(f"❌ Error during training: {e}")
#         raise

# if __name__ == "__main__":
#     main()

In [ ]:
# model_training_fixed.py
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import pickle
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("🚀 Starting Model Training with Streamlit-Compatible Features...")


In [ ]:
# Load your data
try:
    df = pd.read_csv("accident-project.csv")
    print(f"✅ Data loaded successfully! Shape: {df.shape}")
except FileNotFoundError:
    print("❌ File 'accident-project.csv' not found!")
    exit()

# Basic cleaning
df.columns = df.columns.str.strip()
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
df = df.replace('na', np.nan)
df = df.replace('unknown', np.nan)

# Handle missing target
target_col = "Accident_severity"
if target_col not in df.columns:
    print(f"❌ Target column '{target_col}' not found!")
    exit()

df = df[~df[target_col].isna()].copy()
df = df.drop_duplicates()

print(f"📊 Dataset after cleaning: {df.shape}")
print(f"🎯 Target distribution:\n{df[target_col].value_counts()}")

# Define ONLY the features we'll use in Streamlit (matching your sidebar inputs)
streamlit_features = [
    'Hour_of_Day', 'Time_of_Day', 'Age_band_of_driver', 'Sex_of_driver',
    'Educational_level', 'Driving_experience', 'Type_of_vehicle',
    'Area_accident_occured', 'Road_surface_conditions', 'Light_conditions',
    'Weather_conditions', 'Number_of_vehicles_involved', 'Number_of_casualties',
    'Risk_Score', 'Day_of_week', 'Service_year_of_vehicle', 'Experience_Level'
]

# Feature engineering to create the features we need
if 'Time' in df.columns:
    try:
        df['Hour_of_Day'] = pd.to_datetime(df['Time'], errors='coerce').dt.hour.fillna(12)
        df['Time_of_Day'] = pd.cut(df['Hour_of_Day'], 
                                 bins=[0, 6, 12, 18, 24], 
                                 labels=['Night', 'Morning', 'Afternoon', 'Evening'])
    except:
        df['Hour_of_Day'] = 12
        df['Time_of_Day'] = 'Afternoon'

# Create Risk_Score
if 'Number_of_vehicles_involved' in df.columns and 'Number_of_casualties' in df.columns:
    df['Risk_Score'] = (df['Number_of_vehicles_involved'] + df['Number_of_casualties']) / 2
else:
    df['Risk_Score'] = 1.0

# Create Experience_Level
if 'Driving_experience' in df.columns:
    df['Experience_Level'] = df['Driving_experience'].map({
        'No Licence': 'Novice',
        'Below 1yr': 'Novice', 
        '1-2yr': 'Beginner',
        '2-5yr': 'Intermediate',
        '5-10yr': 'Experienced',
        'Above 10yr': 'Expert',
        'Unknown': 'Intermediate'
    }).fillna('Intermediate')

# Fill missing values for essential features
default_values = {
    'Day_of_week': 'Monday',
    'Service_year_of_vehicle': '5-10yr',
    'Age_band_of_driver': '18-30',
    'Sex_of_driver': 'Male',
    'Educational_level': 'High school',
    'Driving_experience': '2-5yr',
    'Type_of_vehicle': 'Automobile',
    'Area_accident_occured': 'Residential areas',
    'Road_surface_conditions': 'Dry',
    'Light_conditions': 'Daylight',
    'Weather_conditions': 'Normal',
    'Number_of_vehicles_involved': 2,
    'Number_of_casualties': 1
}

for col, default_val in default_values.items():
    if col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].fillna(default_val)
        else:
            df[col] = df[col].fillna(default_val)

# Select only the features we'll use in Streamlit
available_features = [f for f in streamlit_features if f in df.columns]
print(f"🔧 Using features: {available_features}")

# Define feature types
numeric_features = ['Hour_of_Day', 'Number_of_vehicles_involved', 'Number_of_casualties', 'Risk_Score']
numeric_features = [f for f in numeric_features if f in available_features]

categorical_features = [f for f in available_features if f not in numeric_features and f != target_col]

# Define ordinal features
ordinal_mappings = {
    'Day_of_week': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    'Age_band_of_driver': ['Under 18', '18-30', '31-50', 'Over 51'],
    'Educational_level': ['Elementary school', 'Junior high school', 'High school', 'Above high school'],
    'Driving_experience': ['Below 1yr', '1-2yr', '2-5yr', '5-10yr', 'Above 10yr'],
    'Service_year_of_vehicle': ['Below 1yr', '1-2yr', '2-5yr', '5-10yr', 'Above 10yr'],
    'Experience_Level': ['Novice', 'Beginner', 'Intermediate', 'Experienced', 'Expert'],
    'Time_of_Day': ['Night', 'Morning', 'Afternoon', 'Evening']
}

ordinal_cols = [col for col in ordinal_mappings.keys() if col in categorical_features]
ordinal_categories = [ordinal_mappings[col] for col in ordinal_cols]
nominal_cols = [col for col in categorical_features if col not in ordinal_cols]

print(f"📊 Numeric features: {numeric_features}")
print(f"📊 Ordinal features: {ordinal_cols}")
print(f"📊 Nominal features: {nominal_cols}")

# Create preprocessing pipeline
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

ordinal_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(categories=ordinal_categories, 
                              handle_unknown='use_encoded_value', unknown_value=-1))
])

nominal_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('ord', ordinal_transformer, ordinal_cols),
        ('cat', nominal_transformer, nominal_cols)
    ],
    remainder='drop'
)

# Prepare target
le_target = LabelEncoder()
y = le_target.fit_transform(df[target_col])
X = df[available_features]  # Use only the features we'll have in Streamlit

target_mapping = dict(zip(le_target.classes_, le_target.transform(le_target.classes_)))
print(f"🎯 Target classes: {target_mapping}")

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"📊 Training set: {X_train.shape}, Test set: {X_test.shape}")

# Train model
print("\n🔮 Training Random Forest Model...")
model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', model)
])

# Train
pipeline.fit(X_train, y_train)

# Evaluate
y_pred = pipeline.predict(X_test)

metrics = {
    'Accuracy': accuracy_score(y_test, y_pred),
    'Precision': precision_score(y_test, y_pred, average='weighted', zero_division=0),
    'Recall': recall_score(y_test, y_pred, average='weighted', zero_division=0),
    'F1-Score': f1_score(y_test, y_pred, average='weighted', zero_division=0)
}

print(f"📊 Model Performance:")
for metric, value in metrics.items():
    print(f"   {metric}: {value:.4f}")

# Save model artifacts
model_artifacts = {
    'model': pipeline,
    'preprocessor': preprocessor,
    'label_encoder': le_target,
    'target_mapping': target_mapping,
    'feature_names': available_features,  # Store the exact features we used
    'model_performance': {'Random Forest': metrics},
    'best_model_name': 'Random Forest',
    'required_features': available_features  # Explicitly store required features
}

with open('accident_severity_model.pkl', 'wb') as f:
    pickle.dump(model_artifacts, f)

# Save metadata
metadata = {
    'model_info': {
        'best_model': 'Random Forest',
        'performance': metrics,
        'target_classes': le_target.classes_.tolist(),
        'feature_count': len(available_features),
        'required_features': available_features
    },
    'training_date': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S'),
    'dataset_info': {
        'total_samples': len(df),
        'training_samples': len(X_train),
        'test_samples': len(X_test)
    }
}

with open('model_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("\n✅ Model training completed successfully!")
print(f"💾 Model saved with {len(available_features)} features")
print("🚀 Next: Run 'streamlit run streamlit_app.py'")